In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import os.path
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model
import keras


Using TensorFlow backend.


In [4]:
# create the base pre-trained model
base_model = keras.applications.DenseNet201(weights='imagenet', include_top=False)

# dimensions of our images.
#Inception input size
img_width, img_height = 299, 299

top_layers_checkpoint_path = 'cp_1.top.best.hdf5'
fine_tuned_checkpoint_path = 'cp.fine_tuned.best.hdf5'
new_extended_inception_weights = 'final_weights.hdf5'

train_data_dir = '/home/ozgu/Desktop/DATASETS/UCMerced_LandUse/Images/'
validation_data_dir = ''

nb_train_samples = 2100
nb_validation_samples = 0

top_epochs = 150
fit_epochs = 150

batch_size = 16

ValueError: You are trying to load a weight file containing 532 layers into a model with 526 layers.

In [13]:
#add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- we have 2 classes
predictions = Dense(21, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

if os.path.exists(top_layers_checkpoint_path):
	model.load_weights(top_layers_checkpoint_path)
	print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], )

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

#validation_generator = test_datagen.flow_from_directory(
#    validation_data_dir,
#   target_size=(img_height, img_width),
#    batch_size=batch_size,
#    class_mode='categorical')


#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)



#Save the TensorBoard logs.
tb = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)

# train the model on the new data for a few epochs
#model.fit_generator(...)

#model.fit_generator(
#    train_generator,
#    samples_per_epoch=nb_train_samples // batch_size,
#    nb_epoch=top_epochs,
#    validation_data=validation_generator,
#    nb_val_samples=nb_validation_samples // batch_size)


Found 2100 images belonging to 21 classes.


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
#transfer learning for all networ, all layers!
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs)

mc_fit = ModelCheckpoint(fine_tuned_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


if os.path.exists(fine_tuned_checkpoint_path):
	model.load_weights(fine_tuned_checkpoint_path)
	print ("Checkpoint '" + fine_tuned_checkpoint_path + "' loaded.")

model.save_weights(new_extended_inception_weights)

model.summary()

In [14]:
# we chose to train the top 2 inception blocks, i.e. we will freeze the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again
# alongside the top Dense layers
#model.fit_generator(...)
''' if validation data exist
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=fit_epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples // batch_size)
'''
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=fit_epochs)

model.save_weights(new_extended_inception_weights)

model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=250, steps_per_epoch=8)`


Epoch 1/250
8/8 [==============================] - 21s 3s/step - loss: 3.2109 - acc: 0.0391
Epoch 2/250
8/8 [==============================] - 2s 244ms/step - loss: 3.1429 - acc: 0.0078
Epoch 3/250
8/8 [==============================] - 2s 250ms/step - loss: 3.0843 - acc: 0.0703
Epoch 4/250
8/8 [==============================] - 2s 232ms/step - loss: 3.0507 - acc: 0.0469
Epoch 5/250
8/8 [==============================] - 2s 231ms/step - loss: 2.9742 - acc: 0.0938
Epoch 6/250
8/8 [==============================] - 2s 231ms/step - loss: 2.9422 - acc: 0.1094
Epoch 7/250
8/8 [==============================] - 3s 387ms/step - loss: 2.9495 - acc: 0.1027
Epoch 8/250
8/8 [==============================] - 2s 232ms/step - loss: 2.7557 - acc: 0.1562
Epoch 9/250
8/8 [==============================] - 2s 231ms/step - loss: 2.7553 - acc: 0.2266
Epoch 10/250
8/8 [==============================] - 2s 231ms/step - loss: 2.7551 - acc: 0.1797
Epoch 11/250
8/8 [==============================] - 2s 234ms/

8/8 [==============================] - 2s 224ms/step - loss: 0.2510 - acc: 0.9688
Epoch 173/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2372 - acc: 0.9688
Epoch 174/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2687 - acc: 0.9453
Epoch 175/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2731 - acc: 0.9531
Epoch 176/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2261 - acc: 0.9922
Epoch 177/250
8/8 [==============================] - 2s 225ms/step - loss: 0.2669 - acc: 0.9609
Epoch 178/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2146 - acc: 0.9844
Epoch 179/250
8/8 [==============================] - 2s 225ms/step - loss: 0.2895 - acc: 0.9688
Epoch 180/250
8/8 [==============================] - 2s 224ms/step - loss: 0.2526 - acc: 0.9609
Epoch 181/250
8/8 [==============================] - 2s 223ms/step - loss: 0.2463 - acc: 0.9609
Epoch 182/250
8/8 [==============================] - 2

In [15]:
extractLayerName = "dense_15"

model_new = Model(inputs=model.input, outputs=model.get_layer(extractLayerName).output)

In [16]:
import glob
import numpy as np
from keras.applications.resnet50 import preprocess_input, decode_predictions
#feature extraction steps
feature_list = [] 
feature_list = glob.glob("/home/ozgu/Desktop/DATASETS/aid_all/*.jpg")
print(feature_list[0])
print(len(feature_list))
img_path = '/home/ozgu/Desktop/DATASETS/AID/Airport/airport_1.jpg'
for i in range(len(feature_list)):
    img_path = feature_list[i] 
    #print(i)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model_new.predict(x)
    np.savetxt(feature_list[i] + ".txt",preds)

print("Done...")

/home/ozgu/Desktop/DATASETS/aid_all/airport_302.jpg
10000
Done...
